<a href="https://colab.research.google.com/github/Gasthony/animedays/blob/main/Trading_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Introducción al Trading Bot de Criptomonedas**

*Objetivo y Alcance del Proyecto*

Este proyecto se enfoca en el diseño y la implementación de un bot de trading automático para criptomonedas, específicamente para la moneda Bitcoin (BTC) contra el dólar estadounidense (USDT). El objetivo principal es aplicar técnicas avanzadas de análisis de datos y estrategias de trading algorítmico para facilitar decisiones de inversión automatizadas. El bot está diseñado para operar en la plataforma Binance, una de las bolsas de criptomonedas más grandes del mundo, integrando diversas librerías como ccxt para la conexión con la API de Binance, pandas y SQLAlchemy para la manipulación de datos, y TA-Lib para el análisis técnico. Este enfoque práctico al trading algorítmico no solo permite aplicar conocimientos adquiridos en el máster de Data Strategy and Analytics, sino que también abre caminos para futuras investigaciones y desarrollos en el campo de las finanzas cuantitativas.

*Desarrollo y Metodología Implementada*

El bot se ha desarrollado en Python, aprovechando su capacidad para el análisis de datos y la automatización de tareas. La metodología sigue varios pasos clave: en primer lugar, la configuración inicial incluye la instalación de las librerías necesarias y la configuración de las claves API para Binance. Posteriormente, se establece una conexión con la base de datos a través de SQLAlchemy para almacenar los datos en tiempo real. La función principal fetch_ohlcv recopila datos históricos de precios de Bitcoin y calcula varios indicadores técnicos, como el Bolsillo Áureo, el Alligator de Williams, las Bandas de Bollinger y la Nube Ichimoku. Estos indicadores son fundamentales para la generación de señales de trading, que se evalúan en la función check_signals. Si se detecta una señal de compra o venta, el bot envía automáticamente un correo electrónico al usuario con los detalles pertinentes. Este proceso no solo demuestra habilidades en la programación y análisis de datos, sino que también refleja una comprensión profunda de las estrategias de trading y su aplicación práctica.

*Resultados y Aplicaciones Futuras*

El resultado del proyecto es un bot de trading completamente funcional que opera en tiempo real, proporcionando señales de trading basadas en análisis técnico complejo. La implementación de este bot en un entorno de trading real ofrece un campo de prueba invaluable para estrategias de inversión basadas en datos, y su diseño modular permite adaptaciones y mejoras futuras. Además, el proyecto sirve como un caso de estudio excelente para entender cómo la tecnología y los análisis avanzados pueden ser aplicados en el mundo financiero, especialmente en el área emergente de las criptomonedas. Esta experiencia no solo es relevante para el máster, sino que también proporciona una base sólida para carreras en data science, finanzas cuantitativas y trading algorítmico.

In [ ]:
!pip install python-binance pandas sqlalchemy TA-Lib

In [ ]:
!pip install ccxt

In [ ]:
import ccxt
import numpy as np
import talib
import time
import datetime
import os
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import pandas as pd
import sqlalchemy
from binance import BinanceSocketManager, AsyncClient

# Configuramos settings de correo
email_user = 'miotifinal@outlook.es'
email_password = 'Trabajofinal2024'
email_recipient = 'carlosc@alumni.mioti.es'

# Configuramos llaves API de Binance
api_key = 'jB4RKugRkL99znSLmjFqzefR4UVoSmr7KtXWdeq6yl8wZdFijUw0406VFvXukWv7'
api_secret = 'IyNtruM0POE5CQsCmR9OkFcAhysBMgU6L7CuMnyc6GuSMtJWDbF3lS1fIDlHUPpw'

# Creamos Cliente Binance
client = ccxt.binance({
    'apiKey': api_key,
    'secret': api_secret,
    'enableRateLimit': True,
})

# Creamos una SQLAlchemy engine para la base de datos
engine = sqlalchemy.create_engine('sqlite:///BTCUSDTstream.db')

# Función para obtener data OHLCV de Binance a traves de ccxt
def fetch_ohlcv(symbol, timeframe, since):
    ohlcv = client.fetch_ohlcv(symbol, timeframe, since)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

    # Calculos Adicionales
    # Golden Pocket and Alligator Indicator
    df['Máximo_52S'] = df['High'].rolling(window=52).max()
    df['Mínimo_52S'] = df['Low'].rolling(window=52).min()
    df['Proporción_Áurea'] = (df['Máximo_52S'] - df['Mínimo_52S']) * 0.618
    df['Compra_Bolsillo_Áureo'] = df['Low'] <= df['Proporción_Áurea']
    df['Venta_Bolsillo_Áureo'] = df['High'] >= df['Proporción_Áurea']

    # Alligator de Williams
    df['Mandíbula_Alligator'] = talib.SMA(df['Close'], timeperiod=13).shift(8)
    df['Dientes_Alligator'] = talib.SMA(df['Close'], timeperiod=8).shift(5)
    df['Labios_Alligator'] = talib.SMA(df['Close'], timeperiod=5).shift(3)

    # Bandas de Bollinger
    upper_band, middle_band, lower_band = talib.BBANDS(df['Close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
    df['Banda_Superior'] = upper_band
    df['Banda_Media'] = middle_band
    df['Banda_Inferior'] = lower_band

    # Nube Ichimoku
    nine_period_high = df['High'].rolling(window=9).max()
    nine_period_low = df['Low'].rolling(window=9).min()
    df['Span_A_Ichimoku'] = ((nine_period_high + nine_period_low) / 2).shift(26)

    fiftytwo_period_high = df['High'].rolling(window=52).max()
    fiftytwo_period_low = df['Low'].rolling(window=52).min()
    df['Span_B_Ichimoku'] = ((fiftytwo_period_high + fiftytwo_period_low) / 2).shift(26)

    return df

# Función para calculo de señales
def check_signals(df):
    # Example: Check Golden Pocket Buy and Sell Conditions
    signal = None
    if df['Close'].iloc[-1] < df['Proporción_Áurea'].iloc[-1]:
        signal = 'Buy_Golden_Pocket'
    elif df['Close'].iloc[-1] > df['Proporción_Áurea'].iloc[-1]:
        signal = 'Sell_Golden_Pocket'

    # Check Alligator Conditions
    if df['Labios_Alligator'].iloc[-1] > df['Dientes_Alligator'].iloc[-1] and df['Labios_Alligator'].iloc[-1] > df['Mandíbula_Alligator'].iloc[-1]:
        signal = 'Buy_Alligator'
    elif df['Labios_Alligator'].iloc[-1] < df['Dientes_Alligator'].iloc[-1] and df['Labios_Alligator'].iloc[-1] < df['Mandíbula_Alligator'].iloc[-1]:
        signal = 'Sell_Alligator'

    # Check Bollinger Bands Conditions
    if df['Close'].iloc[-1] < df['Banda_Inferior'].iloc[-1]:
        signal = 'Buy_Bollinger'
    elif df['Close'].iloc[-1] > df['Banda_Superior'].iloc[-1]:
        signal = 'Sell_Bollinger'

    # Check Ichimoku Conditions
    if df['Close'].iloc[-1] > df['Span_A_Ichimoku'].iloc[-1] and df['Close'].iloc[-1] > df['Span_B_Ichimoku'].iloc[-1]:
        signal = 'Buy_Ichimoku'
    elif df['Close'].iloc[-1] < df['Span_A_Ichimoku'].iloc[-1] and df['Close'].iloc[-1] < df['Span_B_Ichimoku'].iloc[-1]:
        signal = 'Sell_Ichimoku'

    return signal, df['Close'].iloc[-1]

# Funcion para mandar correos
def send_email(signal, price, timestamp):
    subject = f"Trading Signal: {signal}"

    msg = MIMEMultipart()
    msg['From'] = email_user
    msg['To'] = email_recipient
    msg['Subject'] = subject

    body = f"{timestamp} - {signal} - Price: {price}"
    msg.attach(MIMEText(body, 'plain'))

    try:
        server = smtplib.SMTP('smtp.outlook.com', 587)
        server.starttls()
        server.login(email_user, email_password)  # Ensure correct credentials
        text = msg.as_string()
        server.sendmail(email_user, email_recipient, text)
        server.quit()
        print(f"Email sent: {signal}")
    except Exception as e:
        print(f"Error sending email: {e}")

# Reemplazar 'BTC/USDT', '1d', and '2013-01-01T00:00:00Z' with your desired symbol, timeframe, and start date
symbol = 'BTC/USDT'
timeframe = '1d'
since = client.parse8601('2013-01-01T00:00:00Z')

# Obtener data OHLCV del intercambio
datos_btc = fetch_ohlcv(symbol, timeframe, since)

# Binance WebSocket streaming code
async def createframe(msg):
    df = pd.DataFrame([msg])
    df = df.loc[:, ['s', 'E', 'p']]
    df.columns = ['Symbol', 'Time', 'Price']
    df.Price = df.Price.astype(float)
    df.Time = pd.to_datetime(df.Time, unit='ms')
    return df

async def main():
    client = await AsyncClient.create(api_key, api_secret)
    bsm = BinanceSocketManager(client)
    socket = bsm.trade_socket('BTCUSDT')

    async with socket as tscm:
        while True:
            msg = await tscm.recv()
            frame = await createframe(msg)
            frame.to_sql('BTCUSDT', engine, if_exists='append', index=False)
            print(msg)

# Specificamos path for documento de salida
output_file_path = 'C:\\Users\\Usuario\\Downloads\\Trabajo Final\\Trading Bot\\Trading Bot.txt'

# Bucle para verificar las señales en tiempo real
while True:
    signal, price = check_signals(datos_btc)
    current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    if signal:
        print(f"{current_time} - {signal} - Precio: {price}")
        send_email(signal, price, current_time)
    else:
        print(f"{current_time} - Sin señal - Precio: {price}")

    with open(output_file_path, "a") as file:
        file.write(f"{current_time} - {signal} - Precio: {price}\n")

    time.sleep(15)



2024-01-28 23:50:39 - Sell_Ichimoku - Precio: 3695.32
Email sent: Sell_Ichimoku
